In [1]:
# 🧱 Install YOLOv5 inside YOLOv11 (as in your repo)
!git clone https://github.com/ultralytics/yolov5  # clone latest YOLOv5
%cd yolov5
%pip install -r requirements.txt

# 🧰 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Cloning into 'yolov5'...
remote: Enumerating objects: 17485, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 17485 (delta 81), reused 31 (delta 31), pack-reused 17374 (from 3)
Receiving objects: 100% (17485/17485), 16.39 MiB | 6.45 MiB/s, done.
Resolving deltas: 100% (11986/11986), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s

In [2]:

import os
DATA_DIR = "/content/drive/MyDrive/PlantDec"
assert os.path.exists(DATA_DIR), "Dataset path not found. Fix your DATA_DIR."

# 🔧 Check YOLO-compatible structure
!tree {DATA_DIR} -L 2


/bin/bash: line 1: tree: command not found


In [3]:
# 👁️ Disease-specific bounding box visualizer
import cv2
import random
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from PIL import Image

def plot_diseases(image_path, label_path, class_names):
    image = np.array(Image.open(image_path).convert("RGB"))
    h, w, _ = image.shape
    color = (255, 0, 0)

    if not os.path.exists(label_path):
        print(f"⚠️ Label not found: {label_path}")
        return

    with open(label_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            class_id = int(parts[0])
            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:5])
            x1 = int((x_center - bbox_width / 2) * w)
            y1 = int((y_center - bbox_height / 2) * h)
            x2 = int((x_center + bbox_width / 2) * w)
            y2 = int((y_center + bbox_height / 2) * h)
            label = class_names[class_id]
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis("off")
    plt.title("Diseased Region Highlighted")
    plt.show()


In [4]:
import os
from pathlib import Path
import yaml

# Paths
dataset_dir = "/content/drive/MyDrive/PlantDec"
train_images = os.path.join(dataset_dir, "images/train")
val_images = os.path.join(dataset_dir, "images/val")

# Extract class names from one label file
label_dir = os.path.join(dataset_dir, "labels/train")
sample_label_file = next(Path(label_dir).glob("*.txt"), None)

# Auto-detect number of classes from all label files (safer)
class_ids = set()
for label_file in Path(label_dir).glob("*.txt"):
    with open(label_file, "r") as f:
        for line in f:
            if line.strip():
                class_ids.add(int(line.strip().split()[0]))

nc = len(class_ids)
names = [f"class_{i}" for i in range(nc)]  # Placeholder names (can be updated manually)

# Construct data.yaml content
data_yaml = {
    "train": train_images,
    "val": val_images,
    "nc": nc,
    "names": names
}



In [5]:
# 🏷️ Load class names from data.yaml
import yaml

yaml_path = os.path.join(DATA_DIR, "data.yaml")
with open(yaml_path, "r") as stream:
    data_yaml = yaml.safe_load(stream)

class_names = data_yaml['names']
print("Classes:", class_names)


Classes: ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf spot', 'Bell_pepper leaf', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Potato leaf', 'Raspberry leaf', 'Soyabean leaf', 'Soybean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato leaf', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf black rot', 'grape leaf']


In [6]:
# 🏋️ Train the model (you can tune epochs, batch, imgsz)
!python train.py --img 416 --batch 4 --epochs 10 --data /content/drive/MyDrive/PlantDec/data.yaml --weights yolov5s.pt --name plantdec_yolov5s


Streaming output truncated to the last 5000 lines.
        5/9     0.602G   0.006208  0.0006089   0.001971          0        416:   3% 13/499 [00:02<01:32,  5.23it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
        5/9     0.602G   0.005765  0.0005797    0.00183          0        416:   3% 14/499 [00:02<01:32,  5.24it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
        5/9     0.602G   0.009123    0.00105   0.003842          2        416:   3% 15/499 [00:02<01:40,  4.80it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
        5/9     0.602G   0.008553 

In [9]:
# 📈 Evaluate on the validation set and get precision, recall, mAP
!python val.py --weights /content/yolov5/runs/train/plantdec_yolov5s/weights/best.pt --data /content/drive/MyDrive/PlantDec/data.yaml --img 640 --task val


val: data=/content/drive/MyDrive/PlantDec/data.yaml, weights=['/content/yolov5/runs/train/plantdec_yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-418-ga493afe1 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7091035 parameters, 0 gradients, 16.0 GFLOPs
val: Scanning /content/drive/MyDrive/PlantDec/labels/val.cache... 347 images, 342 backgrounds, 0 corrupt: 100% 349/349 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 11/11 [00:06<00:00,  1.67it/s]
                   all        349          7    0.00162       0.25    0.00165   0.000396
       Apple Scab Leaf        349          1          0   

In [10]:
!python detect.py --weights runs/train/plantdec_yolov5s2/weights/best.pt --img 640 --source /content/drive/MyDrive/PlantDec/images/test --save-txt --save-conf --project runs/test_results --name plantdec_test --exist-ok 2>&1 | grep -v "0 labels saved" | sed 's/^/print(\"/' | sed 's/$/\")/' > classify.txt


In [11]:

!cat /content/yolov5/classify.txt



          157 layers, 7091035 parameters, 0 gradients, 16.0 GFLOPs
                    Class     Images  Instances          P          R      mAP50   mAP50-95
                      all        349          7     0.420      0.605     0.462     0.239
          Apple Scab Leaf        349          1     0.380      0.400     0.390     0.190
           Apple rust leaf        349          6     0.790      0.820     0.790     0.410
  Results saved to runs/train/plantdec_yolov5s2